<a href="https://colab.research.google.com/github/nicelah/yt-translate-transcribe/blob/main/whisper_ctranslate2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# Your model will be available in the specified path in your Google Drive

import shutil

# Define the path to the model file in Google Drive
drive_model_path = '/content/drive/MyDrive/whisper-ctranslate2/large-v2'

# Define the destination path in the local instance
local_model_path = '/content/large-v2'


# Copy the model file from Google Drive to the local instance
!cp -r {drive_model_path} {local_model_path} 


Mounted at /content/drive


In [2]:
pip install -U whisper-ctranslate2 yt-dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.4 MB/s eta 0:00:00


In [3]:
import os
import re
import subprocess
from IPython.display import display, FileLink
import selectors
from google.colab import files
from ipywidgets import interactive, widgets, VBox, HBox, Layout
import multiprocessing

# Define a function to run and print commands
def run_command(command_args):
    # Convert the command arguments to string and print the command
    command_str = ' '.join(command_args)
    print(f'Running command: {command_str}')
    # Run the command using subprocess.Popen
    process = subprocess.Popen(command_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # Set stdout and stderr pipes to non-blocking mode
    os.set_blocking(process.stdout.fileno(), False)
    os.set_blocking(process.stderr.fileno(), False)
    # Create a selector to monitor stdout and stderr pipes
    sel = selectors.DefaultSelector()
    sel.register(process.stdout, selectors.EVENT_READ, 'stdout')
    sel.register(process.stderr, selectors.EVENT_READ, 'stderr')
    # Loop to read and print data from stdout and stderr
    while True:
        events = sel.select()
        for key, _ in events:
            data = key.fileobj.readline().rstrip()
            label = key.data
            if data:
                print(f'[{label}] {data}')
        # Check if the process is finished
        if process.poll() is not None:
            break
    # Close the selector
    sel.close()

# Define a function to sanitize filenames
def sanitize_filename(filename):
    # Remove any characters that are not alphanumeric or underscore and replace them with underscore
    sanitized_filename = re.sub(r'[^a-zA-Z0-9_]', '_', filename)
    return sanitized_filename

# Define a function that performs the entire process
def process_video(youtube_link, language, task):
    # Define the output filename for the mp3 file
    video_id = youtube_link.split('watch?v=')[-1]

    # Define the output filename as the video ID
    output_filename = f'{sanitize_filename(video_id)}'

    # Download the mp3 of the YouTube link using yt-dlp tool
    download_command = ['yt-dlp', '-x', '--audio-format', 'mp3','-o', f'{output_filename}.mp3', youtube_link]
    run_command(download_command)

    # Define language code mapping
    language_code_mapping = {
        'English': 'en',
        'Chinese': 'zh',
        'French': 'fr',
        'Spanish': 'es',
        'Korean': 'ko',
        'Japanese': 'ja'
    }

    # Pass the filename of the mp3 into the CLI tool whisper-ctranslate2
    translation_command = [
        'whisper-ctranslate2',
        '--model', 'large-v2',
        '--language', language_code_mapping[language],
        '--task', task,
        '--model_directory', local_model_path,
        f'{output_filename}.mp3'
    ]
    run_command(translation_command)

    # Use files.download to provide a link for the user to download the filename.srt file
    print('Downloading subtitles...')
    files.download(f'{output_filename}.srt')

# Define a global variable to hold the multiprocessing.Process instance
current_process = None

# Define form elements
youtube_link_input = widgets.Text(description='YouTube link:')
language_dropdown = widgets.Dropdown(options=['English', 'Chinese', 'French', 'Spanish', 'Korean', 'Japanese'], description='Language:')
task_dropdown = widgets.Dropdown(options=['transcribe', 'translate'], description='Task:')
submit_button = widgets.Button(description='Submit', layout=Layout(border='1px solid black'))
cancel_button = widgets.Button(description='Cancel', layout=Layout(border='1px solid black'))

# Function to be called when the submit button is clicked
def on_submit(button):
    global current_process
    # Retrieve values from the form widgets
    youtube_link = youtube_link_input.value
    language = language_dropdown.value
    task = task_dropdown.value
    
    # If there is an ongoing process, terminate it before starting a new one
    if current_process and current_process.is_alive():
        current_process.terminate()
    
    # Call the process_video function in a separate process
    current_process = multiprocessing.Process(target=process_video, args=(youtube_link, language, task))
    current_process.start()

# Function to be called when the cancel button is clicked
def on_cancel(button):
    global current_process
    # If there is an ongoing process, terminate it
    if current_process and current_process.is_alive():
        current_process.terminate()
    # Clear the input fields
    youtube_link_input.value = ''
    language_dropdown.value = 'English'
    task_dropdown.value = 'transcribe'

# Bind the on_submit function to the submit button's click event
submit_button.on_click(on_submit)

# Bind the on_cancel function to the cancel button's click event
cancel_button.on_click(on_cancel)

# Display the interactive form along with the submit and cancel buttons
form_items = [youtube_link_input, language_dropdown, task_dropdown, HBox([submit_button, cancel_button])]
form = VBox(form_items)
display(form)


Running command: yt-dlp -x --audio-format mp3 -o https___youtu_be_bYSG_NMy7a0.mp3 https://youtu.be/bYSG-NMy7a0
